# CANAL Parser

## Imports

In [23]:
import numpy as np
import logging, cantools, os, math, re, datetime
from jinja2 import Environment
from canSupportLib import little_endian_mask, little_endian_shift_amounts, big_endian_mask, big_endian_shift_amounts, parse_dbc_files, filter_messages_by_node, get_signal_types

## Config

### Variables

In [24]:
dbc_files = ["../../firmware/dbcs/DEMO_CAN.dbc"]
our_node = "FOO"

### Logger

## Parse DBC Files

In [26]:
can_db = parse_dbc_files(dbc_files)

2024-04-12 10:06:43,226 - root - INFO - adding dbc files (['../../firmware/dbcs/DEMO_CAN.dbc'])
2024-04-12 10:06:43,228 - root - INFO - successfully added dbc (../../firmware/dbcs/DEMO_CAN.dbc)


## Get Message Types

In [27]:
signal_types = get_signal_types(can_db)
temp_signal_types = get_signal_types(can_db, allow_floating_point=False)

## Get Messages for Given Node

In [28]:
rx_msgs, tx_msgs = filter_messages_by_node(can_db.messages, our_node)

2024-04-12 10:06:43,240 - root - INFO - filtered messages by node (FOO) num msgs: rx = 1, tx = 1


## 

In [29]:
def get_masks_shifts(msgs):
    masks_shifts_dict = {}

    for msg in msgs:
        masks_shifts_dict[msg.name] = {}
        for sig in msg.signals:
            if sig.byte_order == 'little_endian':
                print("little")
                mask, num_trailing_zeros = little_endian_mask(sig.length, sig.start)
                shift_amounts = little_endian_shift_amounts(num_trailing_zeros)
            elif sig.byte_order == 'big_endian':
                print("big")
                mask, num_trailing_zeros = big_endian_mask(sig.length, sig.start)
                shift_amounts = big_endian_shift_amounts(num_trailing_zeros)
            else:
                print("ERROR: invalid byte order")
                exit(1)
            masks_shifts_dict[msg.name][sig.name] = ([int(byte) for byte in mask], shift_amounts)
    
    return masks_shifts_dict
        
        
unpack_info = get_masks_shifts(rx_msgs)
pack_info = get_masks_shifts(tx_msgs)

big
big
little
shift amounts: [-4, 4, 12, 20, 28, 36, 44, 52]
little
shift amounts: [-14, -6, 2, 10, 18, 26, 34, 42]
little
shift amounts: [-24, -16, -8, 0, 8, 16, 24, 32]
little
shift amounts: [-34, -26, -18, -10, -2, 6, 14, 22]
little
shift amounts: [-44, -36, -28, -20, -12, -4, 4, 12]
little
shift amounts: [-54, -46, -38, -30, -22, -14, -6, 2]


In [30]:
# out = ""

# for msg in rx_msgs:
#     for sig in msg.signals:
#         mask, shifts = unpacking_info[msg.name][sig.name]
#         for i in range(len(mask)):
#             if mask[i] == 0:
#                 continue
#             if shifts[i] < 0:

In [31]:
def camel_to_snake(text):
  '''
  Converts UpperCamelCase to snake_case.
  '''
  
  s1 = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', text)
  
  return re.sub('([a-z0-9])([A-Z])', r'\1_\2', s1).lower()

def decimal_to_hex(decimal_value):
  '''
  Converts a non-negative decimal integer to a lowercase hexadecimal string.

  Args:
      decimal_value: The non-negative decimal integer to convert.

  Returns:
      The hexadecimal representation of the decimal value as a lowercase string.

  Raises:
      TypeError: If the input is not an integer.
      ValueError: If the input is negative.
  '''

  if not isinstance(decimal_value, int):
    raise TypeError("Input must be an integer")
  if decimal_value < 0:
    raise ValueError("Input must be a non-negative integer")

  hex_digits = "0123456789ABCDEF"
  hex_string = ""
  while decimal_value > 0:
    remainder = decimal_value % 16
    hex_string = hex_digits[remainder] + hex_string
    decimal_value //= 16
    
  return "0x" + hex_string.lower()

In [36]:
# Read the template string from a file
with open('canal_messages.jinja2', 'r') as file:
    canal_messages_template = file.read()
    
with open('canal_msg_registry.jinja2', 'r') as file:
    message_registry_template = file.read()

# Create the environment with trim_blocks and lstrip_blocks settings
env = Environment(trim_blocks=True, lstrip_blocks=True)

# Register the camel_to_snake filter (assuming it's defined elsewhere)
env.filters['camel_to_snake'] = camel_to_snake
env.filters['decimal_to_hex'] = decimal_to_hex

# Load the template from the string content
msgs_template = env.from_string(canal_messages_template)
registry_template = env.from_string(message_registry_template)

# Define your context dictionary
context = {
    'date': datetime.date.today().strftime("%Y-%m-%d"),
    'rx_msgs': rx_msgs,
    'tx_msgs': tx_msgs,
    'signal_types': signal_types,
    'temp_signal_types': temp_signal_types,
    'unpack_info': unpack_info,
    'pack_info': pack_info,
    'bus_name': "Veh"
}

# Render the template with the context
rendered_code = msgs_template.render(**context)

# Write the rendered code to a file
with open('../../firmware/projects/DemoCan/generated/can_messages.h', 'w') as output_file:
    output_file.write(rendered_code)

print(f"Rendered code written to 'can_messages.h'")

rendered_code = registry_template.render(**context)

# Write the rendered code to a file
with open('../../firmware/projects/DemoCan/generated/msg_registry.h', 'w') as output_file:
    output_file.write(rendered_code)

print(f"Rendered code written to 'msg_registry.h'")

Rendered code written to 'can_messages.h'
Rendered code written to 'msg_registry.h'


In [ ]:
def generate_from_jinja2_template(template_path: str, output_path: str, context_dict: dict):
    # Read the template string from a file
    with open(template_path, 'r') as file:
        template_str = file.read()

    # Create the environment with trim_blocks and lstrip_blocks settings
    env = Environment(trim_blocks=True, lstrip_blocks=True)

    # Register the camel_to_snake filter (assuming it's defined elsewhere)
    env.filters['camel_to_snake'] = camel_to_snake
    env.filters['decimal_to_hex'] = decimal_to_hex

    # Load the template from the string content
    template = env.from_string(template_str)
    
    # Render the template with the context
    rendered_code = template.render(**context_dict)

    # Write the rendered code to a file
    with open(output_path, 'w') as output_file:
        output_file.write(rendered_code)

    print(f"Rendered code written to '{output_path}'")